In [1]:
using Distributed
# Script for investigating amplitude dependence of potential
@everywhere using Distributions
@everywhere using Test
@everywhere using StatsBase
@everywhere using BenchmarkTools
@everywhere using LinearAlgebra
@everywhere using LaTeXStrings
using Primes
using MCMCDiagnostics
using SharedArrays
using Plots
pyplot()

# Hack for fixing remote channels in 1.0.3 commit 099e826241
@everywhere struct Hack end
function fixRC()
    for p in workers()
        @fetchfrom p Hack()
    end
end

fixRC (generic function with 1 method)

In [2]:
@everywhere src_path = "../Source/"
@everywhere include(src_path*"types.jl")
@everywhere include(src_path*"functions_msc.jl")
@everywhere include(src_path*"functions_neighbors.jl")
@everywhere include(src_path*"functions_types.jl")
@everywhere include(src_path*"functions_energy.jl")
@everywhere include(src_path*"functions_mc.jl")
@everywhere include(src_path*"functions_thermalization.jl")
@everywhere include(src_path*"functions_observables.jl")
@everywhere include(src_path*"functions_symmetries.jl")
include(src_path*"functions_plots_and_files.jl")

makeDirRec (generic function with 1 method)

In [3]:
@everywhere mutable struct Replica
    ψ::State
    state::Int64
    En::Float64
    sim::Controls
end

In [4]:
A = [1,2,3,4]
p = sortperm(A)
println(p)

[1, 2, 3, 4]


In [5]:
function Replica(ψ::State, sim::Controls)
    Replica(ψ, 3, E(ψ), sim)
end

Replica

# Parallelizing Monte-Carlo Sweeps across multiple replicas

The challenge here is to make it so that replicas are not continuously copied to new processes when this needs to be done.

In [6]:
# Make test state.

g = 1.0    # Gauge coupling
ν = 0.3    # Anisotropy
# Other parameters
L = 24     # System length
L₃ = 24
N = L^2*L₃
T_list = [1.64, 1.63, 1.6, 1.61, 1.62, 1.65]
κ₅ = 1.0
f = 0.0/L
syst_list = [SystConstants(L, L₃, 1/g^2, ν, κ₅, f, 1/temp) for temp in T_list]
ψ_list = [State(1, syst; u⁺=1.0, u⁻=0.0) for syst in syst_list]
sim_list = [Controls() for i = 1:length(T_list)]
#pt = PTRun(ψ_list, sim_list, 1; verbose=true)
rep_list = [Replica(ψ_list[i], sim_list[i]) for i = 1:length(ψ_list)];
#println(pt.N_temp)
#println("Temperatures: $([1/R.ψ.consts.β for R in pt.rep_list])")

In [7]:
@everywhere function nMCS!(R::Replica, n::Int64)
    En = R.En
    for i=1:n
        En += mcSweepEn!(R.ψ,R.sim)
    end
    R.En = En
    return R
end

# Fast implementation of distributed lists
We create a new type that takes a normal 1-d Array of any type, distributes it evenly on available processors, and then enables fast parallel processing of the elements in the array, without the need of copying the element data to each process for each parallel execution.

In [8]:
range(1; length=10)

1:10

In [9]:
function getChunckRanges(n::T, nw::Int64=nprocs()-1) where T <: Int
    chunck_min, pluss_num = divrem(n, nw)
    chunck_ranges = Array{UnitRange{Int64}, 1}(undef, nw)
    for i = 1:pluss_num
        chunck_ranges[i] = range((i-1)*(chunck_min+1)+1; length=chunck_min+1)
    end
    for i = pluss_num+1:nw
        chunck_ranges[i] = range(pluss_num*(chunck_min+1)+(i-1-pluss_num)*chunck_min+1; length=chunck_min)
    end
    return chunck_ranges
end

getChunckRanges (generic function with 2 methods)

In [10]:
length(workers())

2

In [11]:
typeof(1:5)

UnitRange{Int64}

In [12]:
@everywhere struct DLChunck{T}
    p::Int64
    chan::RemoteChannel{Channel{Array{T,1}}}
    range::UnitRange{Int64}
end

In [13]:
function distribute(r_list::Array{T,1}; pids = workers()) where {T}
    n = length(r_list)
    nw = length(pids)
    chunck_ranges = getChunckRanges(n, nw)
    
    chunck_list = Array{DLChunck, 1}(undef, nw)
    for (i, p) = enumerate(pids)
        chunck_list[i] = DLChunck(p, RemoteChannel(()->Channel{Array{T,1}}(1), p), chunck_ranges[i])
        put!(chunck_list[i].chan, r_list[chunck_ranges[i]])
    end
    chunck_list
end

distribute (generic function with 1 method)

In [14]:
struct DList
    chunck_list::Array{DLChunck, 1}
    elem_pr_chunck::Array{Int64, 1}
    chuncks::Int64
    chunck_map::Array{Tuple{Int64, Int64}, 1}     # Map from the original indices to a tuple consisting of the index
    # of the chunck containing the index as well as the position in the chunck.
end
import Base.length
function length(dlist::DList)
    l = 0
    for i = 1:dlist.chuncks
        l += dlist.elem_pr_chunck[i]
    end
    return l
end
function DList(r_list::Array{T, 1}) where {T}
    chunck_list = distribute(r_list)
    chuncks = length(chunck_list)
    elem_pr_chunck = Array{Int64, 1}(undef, chuncks)
    chunck_map = Array{Tuple{Int64, Int64}, 1}(undef, length(r_list))
    k = 1
    for i = 1:chuncks
        elem_pr_chunck[i] = length(fetch(chunck_list[i].chan))
        for j = 1:elem_pr_chunck[i]
            chunck_map[k] = (i, j)
            k += 1
        end
    end
    DList(chunck_list, elem_pr_chunck, chuncks, chunck_map)
end
@everywhere function dmap(f::Function, chan::RemoteChannel{Channel{Array{T, 1}}}) where {T}
    local_list = fetch(chan)
    [f(el) for el in local_list]
end
function dmap(f::Function, dlist::DList)
    chuncks = dlist.chuncks
    futures = Array{Future, 1}(undef, chuncks)
    
    for (i, ck) = enumerate(dlist.chunck_list)
        futures[i] = @spawnat ck.p dmap(f, ck.chan)
    end
    vcat([fetch(futures[i]) for i = 1:chuncks]...)
end
        

dmap (generic function with 2 methods)

In [15]:
@everywhere function dmutate(f!::Function, channel::RemoteChannel{Channel{Array{T, 1}}}) where {T}
    local_list = take!(channel)
    for el in local_list
        f!(el)
    end
    put!(channel, local_list)
    nothing
end
function dmutate(f!::Function, list::DList)
    chuncks = list.chuncks
    futures = Array{Future, 1}(undef, chuncks)
    
    for (i, ck) = enumerate(list.chunck_list)
        futures[i] = @spawnat ck.p dmutate(f!, ck.chan)
    end
    for i = 1:chuncks
        wait(futures[i])
    end
    nothing
end

dmutate (generic function with 2 methods)

In [16]:
@everywhere function impureMutate(f!::Function, chan::RemoteChannel{Channel{Array{T, 1}}}) where {T}
    local_list = take!(chan)
    res_list = [f!(el) for el in local_list]
    put!(chan, local_list)
    res_list
end
function impureMutate(f!::Function, dlist::DList)
    chuncks = dlist = chuncks
    futures = Array{Future, 1}(undef, chuncks)
    
    for (i, ck) = enumerate(dlist.chunck_list)
        futures[i] = @spawnat ck.p impureMutat(f!, ck.chan)
    end
    vcat([fetch(futures[i]) for i = 1:chuncks]...)
end

impureMutate (generic function with 2 methods)

In [17]:
println("Energies: $([R.En/N for R in rep_list])")

Energies: [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0]


In [18]:
fixRC()
ck_list = distribute(rep_list)
length(fetch(ck_list[1].chan))

3

In [19]:
#@everywhere function nMCS(remote_r_list::RemoteChannel{Channel{Array{Replica,1}}}, n::T) where T <: Int
#    r_list = take!(remote_r_list)
#    for R in r_list
#        nMCS!(R, n)
#    end
#    put!(remote_r_list, r_list)
#    nothing
#end
#function nMCS(dl_list::Array{DLChunck, 1}, n::I) where I <: Int
#    chuncks = length(dl_list)
#    futures = Array{Future, 1}(undef, chuncks)
#    
#    for (i, dl) = enumerate(dl_list)
#        futures[i] = @spawnat dl.p nMCS(dl.chan, n)
#    end
#    for i = 1:chuncks
#        wait(futures[i])
#    end
#    nothing
#end

In [20]:
function localize(chunck_list::Array{DLChunck, 1})
    return vcat([fetch(ck.chan) for ck in chunck_list]...)
end
function localize(dlist::DList)
    localize(dlist.chunck_list)
end

localize (generic function with 2 methods)

In [21]:
dlist = DList(rep_list);

In [22]:
dmutate(R -> nMCS!(R, 4), dlist)

In [23]:
new_rep_list = localize(dlist)
println("Energies: $([R.En/N for R in new_rep_list])")

Energies: [-2.95476, -2.94735, -2.95603, -2.94986, -2.95318, -2.94809]


In [24]:
@everywhere function E(R::Replica)
    R.En
end

In [25]:
@benchmark dmap(E, dlist)

BenchmarkTools.Trial: 
  memory estimate:  11.75 KiB
  allocs estimate:  279
  --------------
  minimum time:     405.908 μs (0.00% GC)
  median time:      433.041 μs (0.00% GC)
  mean time:        498.429 μs (1.17% GC)
  maximum time:     19.250 ms (66.28% GC)
  --------------
  samples:          9922
  evals/sample:     1

# Benchmarking DList

In [26]:
# Make test state.

g = 1.0    # Gauge coupling
ν = 0.3    # Anisotropy
# Other parameters
L = 24     # System length
L₃ = L
N = L^2*L₃
T_list = [T for T in range(1.6; length=64, step=0.001)]#[1.64, 1.63, 1.6, 1.61, 1.62, 1.65]#
κ₅ = 1.0
f = 0.0/L
syst_list = [SystConstants(L, L₃, 1/g^2, ν, κ₅, f, 1/temp) for temp in T_list]
ψ_list = [State(1, syst; u⁺=1.0, u⁻=0.0) for syst in syst_list]
sim_list = [Controls() for i = 1:length(T_list)]
#pt = PTRun(ψ_list, sim_list, 1; verbose=true)
#println(pt.N_temp)
#println("Temperatures: $([1/R.ψ.consts.β for R in pt.rep_list])")

rep_list = [Replica(ψ_list[i], sim_list[i]) for i = 1:length(ψ_list)];
println("Energies $([R.En/N for R in rep_list])")
dr_list = DList(rep_list);

Energies [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0]


In [27]:
@benchmark dmutate(R -> nMCS!(R, 1), dr_list)

BenchmarkTools.Trial: 
  memory estimate:  12.27 KiB
  allocs estimate:  283
  --------------
  minimum time:     1.128 s (0.00% GC)
  median time:      1.187 s (0.00% GC)
  mean time:        1.178 s (0.00% GC)
  maximum time:     1.204 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [28]:
@benchmark length(dlist)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     42.483 ns (0.00% GC)
  median time:      43.176 ns (0.00% GC)
  mean time:        43.257 ns (0.00% GC)
  maximum time:     66.170 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     991

In [29]:
#import Base.getindex
#function getindex(A::DList, i1::Int64)
    

In [30]:
A = [1,23,5]
@which A[1]

getindex(A::Array, i1::Int64) in Base at array.jl:731

In [31]:
println("Energies $(dmap(E, dr_list)./N)")

Energies [-2.85655, -2.84948, -2.8443, -2.85639, -2.85259, -2.84261, -2.85257, -2.83079, -2.85804, -2.85397, -2.84639, -2.84722, -2.84612, -2.85281, -2.84799, -2.85319, -2.84764, -2.85289, -2.84071, -2.84359, -2.83992, -2.85647, -2.84127, -2.84494, -2.84495, -2.83478, -2.85977, -2.83874, -2.84449, -2.84092, -2.85123, -2.84627, -2.84454, -2.83946, -2.84556, -2.8208, -2.84593, -2.83739, -2.86279, -2.84146, -2.84583, -2.83444, -2.82956, -2.84088, -2.83453, -2.83404, -2.83132, -2.83187, -2.83213, -2.84138, -2.83482, -2.84326, -2.84419, -2.84032, -2.83912, -2.84374, -2.82483, -2.83008, -2.83741, -2.83812, -2.8412, -2.83232, -2.83438, -2.83563]


# Old slow naive methods of parallelization

In [32]:
@benchmark mcSweepEn!(rep_list[1].ψ, rep_list[1].sim)

BenchmarkTools.Trial: 
  memory estimate:  2.74 MiB
  allocs estimate:  69120
  --------------
  minimum time:     45.059 ms (0.00% GC)
  median time:      54.572 ms (0.00% GC)
  mean time:        55.000 ms (2.16% GC)
  maximum time:     65.390 ms (14.60% GC)
  --------------
  samples:          91
  evals/sample:     1

In [33]:
@benchmark pmap(R -> nMCS!(R, 1), rep_list)

BenchmarkTools.Trial: 
  memory estimate:  1.85 GiB
  allocs estimate:  87985492
  --------------
  minimum time:     30.535 s (32.32% GC)
  median time:      30.535 s (32.32% GC)
  mean time:        30.535 s (32.32% GC)
  maximum time:     30.535 s (32.32% GC)
  --------------
  samples:          1
  evals/sample:     1

In [34]:
# --------------------------------------------------------------------------------------------------
# Preform nMCS! on a list of replicas (as much as possible in paralllel)
function nMCS!(rep_list::Array{Replica, 1}, n::Int64)
    nw = nprocs()-1
    n_state = length(rep_list)
    
    i = 0 # Index in rep_list of states already updated.
    while i < n_state
        
        worker_jobs = min(nw, n_state-1-i) # Number of needed jobs given to workers
        # Start the max number of workers if that wouldn't be too much.
        work_futures = [Future() for w = 1:worker_jobs]
        
        for w = 1:worker_jobs
            work_futures[w] = @spawn nMCS!(rep_list[i+w], n)
        end
        index = i+worker_jobs+1
        rep_list[index] =  nMCS!(rep_list[index], n)
        
        for w = 1:worker_jobs
            rep_list[i+w] = fetch(work_futures[w])
        end
        
        i += worker_jobs+1
    end
    
    # After this, all states should have been updated
    return
end

nMCS! (generic function with 3 methods)

# Continuing developing parallel tempering type

In [35]:
@everywhere function distributeTemperatures!(chan::RemoteChannel{Channel{Array{Replica, 1}}}, β_list::Array{Float64, 1})
    local_list = take!(chan)
    for (i, β) = enumerate(β_list)
        s = local_list[i].ψ.consts
        new_syst = SystConstants(s.L, s.L₃, s.g⁻², s.ν, s.κ₅, s.f, β)
        local_list[i].ψ.consts = new_syst
    end
    put!(chan, local_list)
    nothing
end
function distributeTemperatures!(dr_list::DList, β_list::Array{Float64, 1})
    futures = Array{Future, 1}(undef, dr_list.chuncks)
    for (i, ck) = enumerate(dr_list.chunck_list)
        futures[i] = @spawnat ck.p distributeTemperatures!(ck.chan, β_list[ck.range])
    end
    for i = 1:dr_list.chuncks
        wait(futures[i])
    end
    nothing
end

distributeTemperatures! (generic function with 2 methods)

In [36]:
# Make test state.

g = 1.0    # Gauge coupling
ν = 0.3    # Anisotropy
# Other parameters
L = 24     # System length
L₃ = L
N = L^2*L₃
T_list = [T for T in range(1.6; length=4, step=0.001)]#[1.64, 1.63, 1.6, 1.61, 1.62, 1.65]#
κ₅ = 1.0
f = 0.0/L
syst_list = [SystConstants(L, L₃, 1/g^2, ν, κ₅, f, 1/temp) for temp in T_list]
ψ_list = [State(1, syst; u⁺=1.0, u⁻=0.0) for syst in syst_list]
sim_list = [Controls() for i = 1:length(T_list)]

rep_list = [Replica(ψ_list[i], sim_list[i]) for i = 1:length(ψ_list)];
println("Energies $([R.En/N for R in rep_list])")
dr_list = DList(rep_list);
β_list = [1.0, 2.0, 3.0, 4.0]
@benchmark distributeTemperatures!(dr_list, β_list)
#new_rep_list = localize(dr_list)
#[1/R.ψ.consts.β for R in new_rep_list]

Energies [-3.0, -3.0, -3.0, -3.0]


BenchmarkTools.Trial: 
  memory estimate:  9.63 KiB
  allocs estimate:  225
  --------------
  minimum time:     365.076 μs (0.00% GC)
  median time:      436.878 μs (0.00% GC)
  mean time:        489.718 μs (1.13% GC)
  maximum time:     34.226 ms (55.57% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [37]:
mutable struct PTRun
    dr_list::DList              # A list of replicas distributed over available processes.
    accepts::Array{Int64,1}     # Number of accepted PT-swaps between i and i+1
    histograms::Array{Array{Int64, 1},1}
    # This is a list of histograms; 3 histograms pr. temperature. The first is the number of
    # up-moving visits, the 2nd is the number of down-moving visits and the third is the number of visits
    # of uninitialized states.
    β_list::Array{Float64, 1}   # List of inverse temperatures
    rep_map::Array{Int64, 1}       # A map from the index of the inverse temp. in β_list to the corresponding index in
    # dr_list that has the replica with the corresponding temperature.
    E_list::Array{Float64, 1}   # List of energies of the replicas. The index in this list should mirror the indices
                                # in dr_list.
    N_mc::Int64                 # Number of MC sweeps to do between each PT step.
    N_pt::Int64                 # Number of PT steps done
    N_temp::Int64               # Number of temperatures.
end
function PTRun(ψ_list::Array{State,1}, sim_list::Array{Controls,1}, N_mc::Int64; verbose=false)
    N = length(ψ_list)
    N == length(sim_list) || throw(error("ERROR: List of states and control constants are not same length"))
    
    # Sort ψ_list according to increasing temperature
    β_list = [ψ.consts.β for ψ in ψ_list]
    perm = sortperm(β_list; rev=true)
    if verbose && perm != [i for i = 1:N]
        println("WARNING: Inserted state list did not have states in increasing temperature")
    end
    β_list = β_list[perm]
    ψ_list = ψ_list[perm]
    sim_list = sim_list[perm]
    
    # Initialize remaining lists.
    accepts = [0 for i = 1:N-1]
    rep_list = [Replica(ψ_list[i], sim_list[i]) for i = 1:N]
    rep_list[1].state = 1    # The lowest temperature replica is moving up
    rep_list[N].state = 2    # The highest temperature replica is moving down.
    rep_map = [i for i = 1:N]
    E_list = [R.En for R in rep_list]
    histograms = [[0,0,0] for i = 1:N]
    dr_list = DList(rep_list)
    return PTRun(dr_list, accepts, histograms, β_list, rep_map, E_list, N_mc, 0, N)
end

PTRun

In [38]:
function incrementHistograms!(pt::PTRun)
    states = dmap(R -> R.state, pt.dr_list)
    for i = 1:pt.N_temp
        histogram = pt.histograms[i]
        replica_state = states[pt.rep_map[i]]
        histogram[replica_state] += 1
    end
end

incrementHistograms! (generic function with 1 method)

In [87]:
pt = PTRun(ψ_list, sim_list, 1; verbose=true)
println(pt.N_temp)
println("Temperatures: $(dmap(R -> 1/R.ψ.consts.β, pt.dr_list))")
for i = 1:100
    incrementHistograms!(pt)
end
println("Num of histograms: $(length(pt.histograms))×3")
pt.histograms[1]

4
Temperatures: [1.6, 1.601, 1.602, 1.603]
Num of histograms: 4×3


3-element Array{Int64,1}:
 100
   0
   0

In [40]:
function ptSwap!(pt::PTRun, i::Int64, j::Int64)
    # i and j refer to the indices in the temperature array. We swap the contents of rep_map at these indices.
    # This means that effectively we swap the states associated with the temperatures at i and j.
    temp = pt.rep_map[j]
    pt.rep_map[j] = pt.rep_map[i]
    pt.rep_map[i] = temp
    
    return nothing
end

ptSwap! (generic function with 1 method)

In [53]:
function attemptSwap(pt::PTRun, i::Int64, j::Int64)
    
    Δβ = pt.β_list[j] - pt.β_list[i]#ψⱼ.consts.β-ψᵢ.consts.β
    ΔE = pt.E_list[pt.rep_map[j]] - pt.E_list[pt.rep_map[i]]#rep_list[j].En - rep_list[i].En
    
    r = 1-rand() # make a number ∈ (0, 1]
    # Same method as metropolis-hasting selection: to select with probability min{1, e^{Δβ⋅ΔE}}
    # We could choose a random number r ∈ [0, 1] and check if it's lower than r <= e^{Δβ⋅ΔE} provided
    # the exponent is negative. Taking the logarithm on both sides we see that in this case then also
    # log(r) <= Δβ⋅ΔE. This has the advantage that since r ∈ (0,1], log(r) will always be negative
    # and thus we always select if Δβ⋅ΔE is positive, as we should.
    if log(r) <= Δβ*ΔE
        ptSwap!(pt, i, j)
        return true
    end
    return false
end

attemptSwap (generic function with 1 method)

In [42]:
@everywhere function setState(chan::RemoteChannel{Channel{Array{Replica, 1}}}, i::Int64, state::Int64)
    rep_list = take!(chan)
    rep_list[i].state = state
    put!(chan, rep_list)
    nothing
end
function setStates(dr_list::DList, up_index::Int64, down_index::Int64)
    up_tup = dr_list.chunck_map[up_index]
    up_ck = dr_list.chunck_list[up_tup[1]]
    up_fut = @spawnat up_ck.p setState(up_ck.chan, up_tup[2], 1)
    
    down_tup = dr_list.chunck_map[down_index]
    down_ck = dr_list.chunck_list[down_tup[1]]
    down_fut = @spawnat down_ck.p setState(down_ck.chan, down_tup[2], 2)
    
    wait(up_fut)
    wait(down_fut)
    nothing
end

setStates (generic function with 1 method)

In [43]:
setStates(pt.dr_list, 4, 1)
[R.state for R in localize(pt.dr_list)]

4-element Array{Int64,1}:
 2
 3
 3
 1

In [44]:
@benchmark setStates(pt.dr_list, 1, 2)

BenchmarkTools.Trial: 
  memory estimate:  9.56 KiB
  allocs estimate:  228
  --------------
  minimum time:     400.254 μs (0.00% GC)
  median time:      418.671 μs (0.00% GC)
  mean time:        492.079 μs (0.65% GC)
  maximum time:     42.884 ms (23.94% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [81]:
function PTStep!(pt::PTRun)
    # First we preform N_mc MC sweeps for all of the replicas
    N_mc = pt.N_mc
    dmutate(R -> nMCS!(R, N_mc), pt.dr_list)
    # Get energies
    pt.E_list = dmap(R -> R.En, pt.dr_list)
    
    # Before a squence of swap moves we increment the histograms
    incrementHistograms!(pt)
    
    # Then we go through the temperatures and make a swap with probability
    # p = min{1, e^(-Δβ*ΔE)} between neighboring temperatures.
    for i = 1:pt.N_temp-1
        if attemptSwap(pt, i, i+1)
            # Updates acceptance rates if successful
            pt.accepts[i] += 1
        end
    end
    
    # Distribute the new temperatures to worker processes
    distributeTemperatures!(pt.dr_list, pt.β_list[pt.rep_map[pt.rep_map]])
    
    # Finally increment number of PT steps done and set the end states
    # Since the lowest temperature is at index 1, we set the replica at rep_map[1] to be a up-mover.
    # The temperature at N_temp is highest, thus the replica at rep_map[N_temp] is set to a down-mover.
    pt.N_pt += 1
    setStates(pt.dr_list, pt.rep_map[1], pt.rep_map[pt.N_temp])
    
    nothing
end

PTStep! (generic function with 1 method)

# Testing out PTRun

In [90]:
pt = PTRun(ψ_list, sim_list, 1; verbose=true)
println("Temperatures: $(dmap(R -> 1/R.ψ.consts.β, pt.dr_list))")
println("Energies: $(dmap(R -> R.En/N, pt.dr_list))")

Temperatures: [1.6, 1.601, 1.602, 1.603]
Energies: [-3.0, -3.0, -3.0, -3.0]


In [49]:
N_mc = pt.N_mc

1

In [56]:
@benchmark dmutate(R -> nMCS!(R, N_mc), pt.dr_list)

BenchmarkTools.Trial: 
  memory estimate:  11.44 KiB
  allocs estimate:  255
  --------------
  minimum time:     84.298 ms (0.00% GC)
  median time:      113.285 ms (0.00% GC)
  mean time:        111.972 ms (0.00% GC)
  maximum time:     119.572 ms (0.00% GC)
  --------------
  samples:          45
  evals/sample:     1

In [91]:
for i = 1:500
    PTStep!(pt)
end

In [92]:
@benchmark PTStep!(pt)

BenchmarkTools.Trial: 
  memory estimate:  59.08 KiB
  allocs estimate:  1348
  --------------
  minimum time:     84.192 ms (0.00% GC)
  median time:      117.843 ms (0.00% GC)
  mean time:        114.702 ms (0.00% GC)
  maximum time:     133.167 ms (0.00% GC)
  --------------
  samples:          44
  evals/sample:     1

In [97]:
[h[1] for h in pt.histograms]

4-element Array{Int64,1}:
 591
  71
  45
   0

In [99]:
pt.histograms

4-element Array{Array{Int64,1},1}:
 [591, 0, 0] 
 [71, 518, 2]
 [45, 545, 1]
 [0, 591, 0] 

In [100]:
pt.N_pt

591

In [101]:
function getARList(pt::PTRun)
    return [an/(pt.N_pt) for an in pt.accepts]
end

getARList (generic function with 1 method)

In [102]:
function getHistograms(pt::PTRun)
    return [hist[1]/(hist[1]+hist[2]) for hist in pt.histograms]
end

getHistograms (generic function with 1 method)

In [103]:
getHistograms(pt)

4-element Array{Float64,1}:
 1.0                
 0.12054329371816638
 0.07627118644067797
 0.0                

In [111]:
floor.(Int64, round.(getARList(pt).*100; digits=0))

3-element Array{Int64,1}:
 94
 95
 92